In [1]:
import brainpy as bp
import brainpy.math as bm
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5,6"  # specify which GPU(s) to be used
bm.disable_gpu_memory_preallocation()
bm.set_platform('gpu')

/home/spirit/anaconda3/envs/slca/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
s =bm.array([0.5,1,1.5])
phi = bm.array([[0.3313,0.8148,0.4364],[0.8835,0.3621,0.2182],[0.3313,0.4527,0.8729]])
b = bm.dot(phi.T,s)
w = bm.dot(phi.T,phi)
w[bm.diag_indices_from(w)] = 0
print(w)
print(b)

Array(value=Array([[0.        , 0.7397555 , 0.62646115],
                   [0.7397555 , 0.        , 0.82969487],
                   [0.62646115, 0.82969487, 0.        ]]),
      dtype=float32)
Array(value=Array([1.5460999, 1.44855  , 1.74575  ]), dtype=float32)


In [8]:
class Spiking_LCA(bp.DynamicalSystemNS):
    def __init__(self, num, w, b, lamb, method='rk2'):
        super(Spiking_LCA, self).__init__()
        
        self.lamb = lamb
        
        # network size
        num_neuron = int(num)
        self.N = bp.neurons.GIF(num_neuron, method=method)
        self.average_current = bm.Variable(bm.zeros(num_neuron))          # 平均电流记录
        # synapses
        self.N2N = bp.synapses.Exponential(pre=self.N,
                                            post=self.N,
                                            conn= bp.connect.All2All(),
                                            g_max= w,
                                            tau=1.,
                                            output=bp.synouts.CUBA(),
                                            method=method)

    def reversed_function(self, x):
        return -1.87392121 * x**5 + 6.91853151 * x**4 - 9.31760692 * x**3 + 5.73923156 * x**2 + 18.46991473 * x + 0.66899093


    # def reversed_function(self, x):
    #     return -0.0471247 * x**3 + 0.32773097 * x**2 + 2.83983869 * x + 0.20385288

                
    def update(self):
        t = bp.share.load('t')
        self.N2N()
        self.average_current.value = self.average_current *t/(t + 1) + (b - self.N2N.g)/(t+1)
        firing_rate  =  bm.maximum(self.average_current - self.lamb, 0.)
        self.N.input =  bm.where(firing_rate > 1e-7, self.reversed_function(firing_rate) , 1.)
        self.N()

In [9]:
net    = Spiking_LCA(w.shape[0],w,b,0.1)
total_period = 3000
runner = bp.DSRunner(net,monitors=['N.V','N.spike'], dt = 0.01)
runner.run(total_period)

Predict 300000 steps: :   0%|          | 0/300000 [00:00<?, ?it/s]

Predict 300000 steps: : 100%|██████████| 300000/300000 [02:24<00:00, 2073.67it/s]


In [10]:
size_num = runner.mon['N.spike'].shape[0]
print(size_num)
spike_calculate = runner.mon['N.spike'][size_num//4:,:]
sum_along_columns = np.sum(spike_calculate, axis=0)/total_period*4/3
print(sum_along_columns)

300000
[0.68       0.         1.21955556]
